In [66]:
import numpy as np
import pytest
from sklearn.linear_model import Ridge, LinearRegression
from abc import ABC, abstractmethod

In [67]:
class Regr(ABC):
    def _get_X(self,X):
        self.__n = X.shape[0]
        self.X_extended = np.vstack((np.ones(self.__n),X.T))
        
    @abstractmethod
    def fit(self):
        pass
    
    def predict(self, X):
        k, m = X.shape
        self.predicted_values = X@self.theta[1:] + self.theta[0]
        return self.predicted_values
    
class LinearRegr(Regr):
    def fit(self, X, Y):
        super()._get_X(X)
        self.theta = np.linalg.inv(self.X_extended@self.X_extended.T)@self.X_extended@Y
        return self

class RidgeRegr(Regr):
    def __init__(self, alpha = 0.0, c = 0.005, eps = 0.0000001, max_iter = 10000):
        self.alpha = alpha
        self.eps = eps
        self.c = c
        self.max_iter = max_iter
        
    def fit(self, X, Y):
        self.__m = X.shape[1]
        super()._get_X(X)
        self.theta = np.ones(self.__m+1)
        grad = np.ones(self.__m+1)
        a = self.X_extended.T
        for _ in range(self.max_iter):
            for i in range(self.__m+1):
                if i == 0:
                    grad[i] = -2*(a[:,i]@(Y - a@self.theta))
                else:
                    grad[i] = -2*(a[:,i]@(Y - a@self.theta)) + 2*self.alpha*self.theta[i]
            self.theta = self.theta - self.c*grad
        return self    

In [68]:
class Test:
    def __init__(self):
        self.X = np.array([1,3,2,5]).reshape((4,1))
        self.X_ext = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
        self.Y = np.array([2,5, 3, 8])
                
    def test_up(self, method1, method2, dim, rel = 0):
        if dim == 3:
            self.X = self.X_ext
        expected = method1.fit(self.X, self.Y).predict(self.X_test)
        actual = method2.fit(self.X, self.Y).predict(self.X_test)
        assert list(actual) == pytest.approx(list(expected), rel=rel)

class TestLinear(Test):
    def test(self, dim = 1):
        self.X_test = np.array([1,2,10]).reshape((3,1))
        if dim == 3:
            self.X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
        return self.test_up(LinearRegression(), LinearRegr(), dim)

class TestRidge(Test):
    def test(self, dim = 1):
        self.X_test = np.array([1,2,10]).reshape((3,1))
        rel = 1e-5
        alpha = 0.3
        if dim == 3:
            self.X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
            rel = 1e-3
            alpha = 0.4
        return self.test_up(Ridge(alpha), RidgeRegr(alpha), dim, rel = rel)

In [69]:
TestLinear().test()

In [70]:
TestLinear().test(dim = 3)

In [71]:
TestRidge().test()

In [72]:
TestRidge().test(dim = 3)